In [1]:
import pandas as pd
from tqdm import tqdm
import vectorbt as vbt
from helpers.vietcap import VietCapClient

### DANH SÁCH MÃ CP THUỘC NHÓM VN400
> VN400 là top 400 mã CP có `ev` (*vốn hóa*) lớn nhất

In [2]:
vcsc = VietCapClient()
# tất cả các mã CP đang hoạt động
df_stocks = vcsc.get_stocks()
df_stocks.shape

(1586, 25)

In [4]:
# top 200 mã CP có vốn hóa lớn nhất
df_vn400 = (
    df_stocks
    .sort_values("ev", ascending=False)
    .head(400)
    .sort_values("ticker")
    .reset_index(drop=True)
    .loc[:,["ticker","exchange","closePrice","totalVolume","ev","pe","pb","roa","roe"]]
)
df_vn400

,ticker,exchange,closePrice,totalVolume,ev,pe,pb,roa,roe
0,AAA,HOSE,8200,5331904,3.134651e+12,11.847881,0.597148,0.020683,0.050335
1,AAS,UPCOM,15100,1476422,3.567753e+12,54.472181,1.403915,0.014396,0.026069
2,ABB,UPCOM,13500,19781319,1.390779e+13,9.556621,0.907401,0.007979,0.102281
3,ABI,UPCOM,29000,102332,2.100085e+12,10.154915,1.274234,0.046730,0.129071
4,ABW,UPCOM,12400,1066333,1.233727e+12,11.675804,0.823983,0.030384,0.072663
...,...,...,...,...,...,...,...,...,...
395,VSH,HOSE,46600,9122,1.100884e+13,13.512823,2.204151,0.094741,0.169274
396,VSN,UPCOM,17100,8431,1.386784e+12,15.567287,1.128950,0.044323,0.069894
397,VTP,HOSE,104000,849131,1.266544e+13,31.142483,7.548273,0.063203,0.254115
398,VTZ,HNX,19000,1412300,1.447037e+12,15.972451,1.690839,0.029875,0.099639


### LỊCH SỬ GIAO DỊCH
***Lấy dữ liệu lịch sử giao dịch của các mã CP thuộc nhóm VN400***

In [5]:
vcsc = VietCapClient()
# lấy dữ liệu lịch sử từ 2016
from_date = "2016-01-01"
df_historical_price = pd.DataFrame()
for ticker in tqdm(df_vn400["ticker"]):
    # lấy dữ liệu lịch sử từ vcsc
    data = vcsc.get_historical_price(ticker, from_date=from_date)
    # chỉ append nếu có dữ liệu
    if len(data) > 0:
        # chuyển dữ liệu dạng list thành bảng
        df = pd.DataFrame(data)
        # chuyển cột `tradingDate` từ dạng timestamp thành datetime
        df["tradingDate"] = pd.to_datetime(df["tradingDate"], unit="ms")
        # append dữ liệu
        df_historical_price = pd.concat([df_historical_price, df], ignore_index=True)

100%|██████████| 400/400 [04:01<00:00,  1.66it/s]


In [8]:
# dữ liệu giao dịch có nhiều cột
df_historical_price.iloc[0]

tradingDate                 2025-08-20 00:00:00
ticker                                      AAA
openPrice                                  8520
highestPrice                               8520
lowestPrice                                8000
closePrice                                 8200
openPriceAdjusted                        8520.0
highestPriceAdjusted                     8520.0
lowestPriceAdjusted                      8000.0
closePriceAdjusted                       8200.0
totalMatchVolume                        5301865
totalMatchValue                     43876752290
totalDealVolume                           30039
totalDealValue                        264308490
unMatchedBuyTradeVolume               3763413.0
unMatchedSellTradeVolume              3516226.0
totalVolume                             5331904
totalValue                          44141060780
totalBuyTrade                            3675.0
totalBuyTradeVolume                   9065278.0
totalSellTrade                          

In [9]:
# rút ngắn dữ liệu cần thiết cho BACKTESTING gồm giá `OHLC` (điều chỉnh) và KL khớp lệnh `totalMatchVolume`
# tính thêm các chỉ số cần thiết để BACKTESTING
df_historical_price_bt = (
    df_historical_price
    .loc[:,["tradingDate","ticker","closePriceAdjusted"]]
    .rename(columns=lambda x: x.replace("PriceAdjusted", ""))
    .sort_values(["ticker","tradingDate"])
    .reset_index(drop=True)
    .assign(
        SMA10  = lambda df: df.groupby("ticker")["close"].transform(lambda x: x.rolling(10).mean()),
        SMA20  = lambda df: df.groupby("ticker")["close"].transform(lambda x: x.rolling(20).mean()),
        SMA50  = lambda df: df.groupby("ticker")["close"].transform(lambda x: x.rolling(50).mean()),
        SMA100 = lambda df: df.groupby("ticker")["close"].transform(lambda x: x.rolling(100).mean()),
        EMA12  = lambda df: df.groupby("ticker")["close"].transform(lambda x: x.ewm(span=12, adjust=False).mean()),
        EMA26  = lambda df: df.groupby("ticker")["close"].transform(lambda x: x.ewm(span=26, adjust=False).mean()),
        MACD   = lambda df: df["EMA12"] - df["EMA26"],
        MACDSignal = lambda df: df.groupby("ticker")["MACD"].transform(lambda x: x.ewm(span=9, adjust=False).mean())
    )
)
df_historical_price_bt.head()

,tradingDate,ticker,close,SMA10,SMA20,SMA50,SMA100,EMA12,EMA26,MACD,MACDSignal
0,2016-01-07,AAA,7572.440647,NaN,NaN,NaN,NaN,7572.440647,7572.440647,0.000000,0.000000
1,2016-01-08,AAA,7515.073672,NaN,NaN,NaN,NaN,7563.614958,7568.191241,-4.576283,-0.915257
2,2016-01-11,AAA,7916.642494,NaN,NaN,NaN,NaN,7617.926887,7594.002445,23.924442,4.052683
3,2016-01-12,AAA,8203.477367,NaN,NaN,NaN,NaN,7708.011576,7639.148736,68.862841,17.014715
4,2016-01-13,AAA,7974.009469,NaN,NaN,NaN,NaN,7748.934329,7663.953235,84.981095,30.607991


In [10]:
# xây dựng logic tín hiệu MUA
## đặt biến
sma10       = df_historical_price_bt["SMA10"]
sma20       = df_historical_price_bt["SMA20"]
sma50       = df_historical_price_bt["SMA50"]
sma100      = df_historical_price_bt["SMA100"]
macd        = df_historical_price_bt["MACD"]
macd_signal = df_historical_price_bt["MACDSignal"]
sma10_lag1  = df_historical_price_bt.groupby("ticker")["SMA10"].shift(1)
sma20_lag1  = df_historical_price_bt.groupby("ticker")["SMA20"].shift(1)
## SMA10 cắt lên SMA20
sma10_crossover_sma20 = (sma10 > sma20) & (sma10_lag1 <= sma20_lag1)
## SMA20 > SMA50
sma20_over_sma_50 = sma20 > sma50
## SMA50 > SMA100
sma50_over_sma_100 = sma50 > sma100
## MACD > MACDsignal
macd_over_macdsignal = macd > macd_signal
## tín hiệu MUA
buy_signal = sma10_crossover_sma20 & sma20_over_sma_50 & sma50_over_sma_100 & macd_over_macdsignal

In [11]:
# xây dựng logic tín hiệu BÁN
## SMA10 cắt xuống SMA20
sell_signal = (sma10 < sma20) & (sma10_lag1 >= sma20_lag1)

In [12]:
## gán ngược tín hiệu MUA/BÁN vào bảng
df_historical_price_bt["buySignal"]  = buy_signal
df_historical_price_bt["sellSignal"] = sell_signal
df_historical_price_bt.head()

,tradingDate,ticker,close,SMA10,SMA20,SMA50,SMA100,EMA12,EMA26,MACD,MACDSignal,buySignal,sellSignal
0,2016-01-07,AAA,7572.440647,NaN,NaN,NaN,NaN,7572.440647,7572.440647,0.000000,0.000000,False,False
1,2016-01-08,AAA,7515.073672,NaN,NaN,NaN,NaN,7563.614958,7568.191241,-4.576283,-0.915257,False,False
2,2016-01-11,AAA,7916.642494,NaN,NaN,NaN,NaN,7617.926887,7594.002445,23.924442,4.052683,False,False
3,2016-01-12,AAA,8203.477367,NaN,NaN,NaN,NaN,7708.011576,7639.148736,68.862841,17.014715,False,False
4,2016-01-13,AAA,7974.009469,NaN,NaN,NaN,NaN,7748.934329,7663.953235,84.981095,30.607991,False,False


### BACKTESTING

In [13]:
backtesting_results = {}
for ticker, df in tqdm(df_historical_price_bt.groupby("ticker")):
    # dùng cột ngày để làm index cho bảng
    df.set_index("tradingDate", inplace=True)
    # loại bỏ 100 phiên đầu tiên vì tính `SMA100`
    df = df.iloc[99:]
    if len(df) > 100: # chỉ backtesting đối với mã CP từ 200 phiên giao dịch trở lên
        # chạy backtesting bằng `vectorbt`
        pf = vbt.Portfolio.from_signals(
            close=df["close"],
            entries=df["buySignal"],
            exits=df["sellSignal"],
            size=1.0,       # mua toàn bộ vốn cho mỗi lệnh
            fees=0.001,     # phí giao dịch 0.1%
            freq="1D"
        )
        # lưu kết quả backtesting
        backtesting_results[ticker] = pf

100%|██████████| 400/400 [00:05<00:00, 73.19it/s] 


In [14]:
# chuyển kết quả backtesting thành bảng
df_backtesting = (
    pd.concat({ticker: pf.stats() for ticker, pf in backtesting_results.items()}, axis=1)
    .transpose()
)

In [15]:
# lưu kết quả backtesting
end_date = df_backtesting["End"].max().strftime("%Y%m%d")
df_backtesting.to_csv("./results/backtesting_results_vn400_{}.csv".format(end_date))

In [16]:
# các thông tin cần thiết để phân tích kết quả backtesting
df_backtesting_sf = (
    df_backtesting
    .loc[
        :,
        [
            "Start","End","Period", # dữ liệu lịch sử dùng để backtesting
            "Start Value","End Value","Total Return [%]", # đánh giá lợi nhuận
            "Max Drawdown [%]", # đánh giá rủi ro
            "Profit Factor", # đánh giá hiệu quả giao dịch
            "Sharpe Ratio", # đánh giá hiệu quả điều chỉnh rủi ro
        ]
    ]
    .assign(
        Start = lambda df: df["Start"].apply(lambda x: x.date()),
        End = lambda df: df["End"].apply(lambda x: x.date()),
        Period = lambda df: df["Period"].apply(lambda x: pd.Timedelta(days=x.days))
    )
    .map(lambda x: round(x,2) if isinstance(x, float) else x)
)
df_backtesting_sf

,Start,End,Period,Start Value,End Value,Total Return [%],Max Drawdown [%],Profit Factor,Sharpe Ratio
AAA,2016-06-06,2025-08-20,2299 days,100.0,101.33,1.33,23.26,1.05,0.09
AAS,2020-12-10,2025-08-20,1169 days,100.0,27.64,-72.36,73.91,0.00,-1.19
ABB,2021-05-27,2025-08-20,1058 days,100.0,88.79,-11.21,12.65,0.00,-0.98
ABI,2016-06-06,2025-08-20,2304 days,100.0,164.81,64.81,19.78,2.64,0.48
ABW,2023-10-18,2025-08-20,459 days,100.0,100.00,0.00,NaN,NaN,inf
...,...,...,...,...,...,...,...,...,...
VSH,2016-06-06,2025-08-20,2305 days,100.0,100.80,0.80,38.26,1.01,0.10
VSN,2017-03-17,2025-08-20,2107 days,100.0,109.00,9.00,17.15,1.99,0.17
VTP,2019-04-23,2025-08-20,1575 days,100.0,96.79,-3.21,21.98,0.84,0.01
VTZ,2022-04-22,2025-08-20,831 days,100.0,126.89,26.89,21.06,1.95,0.54


### KẾT QUẢ PHÂN TÍCH 
***Nhờ chatGPT đọc kết quả backtesting :)))***

***0. Định nghĩa các chỉ số quan trọng***
- `Total Return [%]`: Tổng lời/lỗ của chiến lược trên mã đó trong giai đoạn test → *Ví dụ*: 120% nghĩa là vốn 100 triệu → cuối cùng thành 220 triệu.
- `Max Drawdown [%]`: Mức sụt giảm lớn nhất từ đỉnh về đáy → Giống như khi đang có lãi nhưng sau đó cổ phiếu rớt mạnh. Nếu Max DD = –40%, tức là có lúc tài khoản mất gần một nửa.
- `Sharpe Ratio`: Đo “lãi trên rủi ro” → Số càng cao thì càng “đáng đồng tiền bát gạo”. >1 coi như ổn, >2 là rất tốt.
- `Profit Factor (PF)`: Tổng lãi / Tổng lỗ → *Ví dụ*, PF = 2 nghĩa là kiếm được gấp đôi số tiền đã thua lỗ. PF < 1 thì hệ thống đang lỗ nhiều hơn lãi.

***1. Rule chấm điểm tổng thể***
- `Total Return [%]`:
    ```python
        if   r["Total Return [%]"] >= 100: score += 2 # thêm 2 điểm
        elif r["Total Return [%]"] >= 50 : score += 1 # thêm 1 điểm
        elif r["Total Return [%]"] <= -20: score -= 1 # trừ  1 điểm
        elif r["Total Return [%]"] <= -40: score -= 2 # trừ  2 điểm
    ```
- `Sharpe Ratio`:
    ```python
        if   r["Sharpe Ratio"] >= 1.0  : score += 2 # thêm 2 điểm
        elif r["Sharpe Ratio"] >= 0.5  : score += 1 # thêm 1 điểm
        elif r["Sharpe Ratio"] <  0    : score -= 1 # trừ  1 điểm
    ```
- `Max Drawdown [%]`:
    ```python
        if   r["Max Drawdown [%]"] <= 20: score += 2 # thêm 2 điểm
        elif r["Max Drawdown [%]"] <= 35: score += 1 # thêm 1 điểm
        elif r["Max Drawdown [%]"] >= 60: score -= 1 # trừ  1 điểm
    ```
- `Profit Factor (PF)`:
    ```python
        if r["Profit Factor (PF)"]   >= 2: score += 1 # thêm 1 điểm
        elif r["Profit Factor (PF)"] <  1: score -= 1 # trừ  1 điểm
    ```

***2. Phân loại chất lượng***
- **Strong**: score > 3
- **Good**: 1 < score <= 3
- **Neutral**: 0 <= score <= 1
- **Weak**: score < 0

***3. Kết luận***
- **Chọn lọc** nhóm **Strong**, lợi nhuận cao, rủi ro thấp, Sharpe tốt
- **Bỏ qua** nhóm **Weak**, lỗ nhiều, rủi ro cao
- **Theo dõi** nhóm **Neutral** để test thêm dữ liệu

In [17]:
def bucket_row(r):
    """Hàm chấm điểm tổng thể"""
    score = 0
    # Return
    tr = r["Total Return [%]"]
    if   tr >= 100: score += 2
    elif tr >=  50: score += 1
    elif tr <= -20: score -= 1
    elif tr <= -40: score -= 2
    # Sharpe
    sr = r["Sharpe Ratio"]
    if pd.notna(sr):
        if   sr >= 1.0: score += 2
        elif sr >= 0.5: score += 1
        elif sr <  0  : score -= 1
    # MaxDD (lower better)
    dd = r["Max Drawdown [%]"]
    if pd.notna(dd):
        if   dd <= 20: score += 2
        elif dd <= 35: score += 1
        elif dd >= 60: score -= 1
    # Profit factor
    pf = r["Profit Factor"]
    if pd.notna(pf):
        if   pf >= 2: score += 1
        elif pf <  1: score -= 1
    return score

In [18]:
# tính toán điểm và phân loại cho tất cả các mã CP
df_backtesting_sf["Quality Score"] = df_backtesting_sf.apply(bucket_row, axis=1)
df_backtesting_sf["Bucket"] = pd.cut(
    df_backtesting_sf["Quality Score"],
    bins=[-10,-1,1,3,100],
    labels=["Weak","Neutral","Good","Strong"]
)
df_backtesting_sf["Bucket"].value_counts().sort_index()

Bucket
Weak       157
Neutral    106
Good        65
Strong      67
Name: count, dtype: int64

In [19]:
# các mã CP triển vọng
## STRONG
## Total Return [%] >= 50
## Sharpe Ratio >= 0.5
## Max Drawdown [%] <= 35
## Profit Factor >= 2
df_backtesting_potentials = df_backtesting_sf[
    (df_backtesting_sf["Bucket"]=="Strong") &
    (df_backtesting_sf["Total Return [%]"] >= 50) &
    (df_backtesting_sf["Sharpe Ratio"] >= 0.5) &
    (df_backtesting_sf["Max Drawdown [%]"] <= 35) &
    (df_backtesting_sf["Profit Factor"] >= 2)
]
print("Số mã CP triển vọng:", len(df_backtesting_potentials))
print("Các mã CP triển vọng:", ", ".join(df_backtesting_potentials.index.tolist()))

Số mã CP triển vọng: 46
Các mã CP triển vọng: ACV, ANV, BCM, BFC, BIC, CKG, CTD, CTR, CTX, DHT, DTD, EVS, FCN, GEG, GVR, HUT, IDI, ITA, KDC, MBB, MTA, NCT, NHA, NHH, ORS, PAC, PDR, PGD, PTB, ROS, SBA, SHS, SSI, STB, TCM, TDC, TIP, TMS, TVS, VCP, VCR, VGI, VIB, VIC, VND, VPB


In [20]:
# các mã CP phải bỏ qua
## WEAK
## Total Return [%] <= -20
## Sharpe Ratio < 0
## Max Drawdown [%] >= 60
## Profit Factor < 1
df_backtesting_ignored = df_backtesting_sf[
    (df_backtesting_sf["Bucket"]=="Weak") &
    (df_backtesting_sf["Total Return [%]"] <= -20) &
    (df_backtesting_sf["Sharpe Ratio"] < 0) &
    (df_backtesting_sf["Max Drawdown [%]"] >= 60) &
    (df_backtesting_sf["Profit Factor"] < 1)
]
print("Số mã CP bỏ qua:", len(df_backtesting_ignored))
print("Các mã CP bỏ qua:", ", ".join(df_backtesting_ignored.index.tolist()))

Số mã CP bỏ qua: 23
Các mã CP bỏ qua: AAS, DCF, DNA, DNW, DSC, HHV, HPW, ICN, LLM, MPC, PDN, PTI, SBM, SID, SJG, SVC, TCB, TMG, TNG, TSJ, VAV, VFC, VLB


In [21]:
# các mã CP theo dõi thêm
## NEUTRAL
df_backtesting_watchlist = df_backtesting_sf[
    (df_backtesting_sf["Bucket"]=="Neutral")
]
print("Số mã CP theo dõi:", len(df_backtesting_watchlist))
print("Các mã CP theo dõi:", ", ".join(df_backtesting_watchlist.index.tolist()))

Số mã CP theo dõi: 106
Các mã CP theo dõi: AAA, ABB, ACB, ACG, AGG, AGR, APG, ASM, BID, BRR, BSR, BTH, BVH, C4G, CC1, CHP, CTG, DBD, DHB, DMC, DNP, DPM, DVP, DXG, ELC, FMC, FTS, GIL, HAG, HCM, HDC, HHC, HHS, HPG, HPX, HTG, IDC, IDP, IFS, IMP, KBC, KHG, KLB, KSB, KSF, L14, LBM, LDG, LIC, LPB, LSG, MCM, MIG, MSB, MSH, MVB, MWG, NCG, NET, NKG, NS2, NVB, NVL, PAN, PAP, PGS, PGV, POW, PPC, PPH, PVP, QNP, RAL, SAS, SCG, SCS, SGB, SJS, SSB, SSH, STG, SZL, TBD, THG, TID, TIN, TLG, TMP, TRA, TRC, VAB, VBB, VCS, VCW, VDS, VEA, VEF, VGG, VGS, VGT, VHM, VIF, VJC, VNB, VSH, VTP


In [22]:
# biểu diễn biểu đồ vốn của 1 mã triển vọng
ignored_ticker = df_backtesting_potentials.index[0]
pf = backtesting_results[ignored_ticker]
print(ignored_ticker)
pf.plot().show()

ACV


In [23]:
# biểu diễn biểu đồ vốn của 1 mã bỏ qua
ignored_ticker = df_backtesting_ignored.index[0]
pf = backtesting_results[ignored_ticker]
print(ignored_ticker)
pf.plot().show()

AAS
